# Исследование векторных представлений эмотиконов

Загрузите корпус текстов, содержащих предложения как со словами, так и эмотиконами. Выведите несколько предложений на экран, убедитесь, что эмотиконы отображаются корректно. Кодировка текста - utf-8.

Разделите предложения по словам, считая эмотикон отдельным словом (нужно подобрать правильный символ-разделитель). В корпусе ровно 1000000 строк.

In [3]:
# Импорт модуля os для работы с операционной системой
import os

# Импорт функции time из модуля time для измерения времени выполнения операций
from time import time

# Импорт библиотеки gdown для скачивания файлов с Google Drive
import gdown

URL='https://drive.google.com/u/1/uc?id=1TPaQTYDpf7p2EklUkLdjK9T5ATztj7oF&export=download'

# Скачивание файла по указанному URL-адресу с помощью gdown
gdown.download(url=URL)

# Команда для распаковки архива corpus.zip в текущую директорию
!unzip corpus.zip

Downloading...
From: https://drive.google.com/u/1/uc?id=1TPaQTYDpf7p2EklUkLdjK9T5ATztj7oF&export=download
To: /content/corpus.zip
100%|██████████| 22.0M/22.0M [00:00<00:00, 66.9MB/s]


Archive:  corpus.zip
replace corpus.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: corpus.txt              


In [4]:
# читаем файл и разбиваем на слова по пробелу.
with open("./corpus.txt" , 'r', encoding='utf-8') as f:
            lines = f.readlines()
            train_data = [line.strip().split() for line in lines]

In [5]:
print(*train_data[:10],sep='\n') # несколько предложений

['they', 'know', 'what', 'i', 'love', '😊']
['lemme', 'get', 'to', 'school', '🙄']
['watching', 'the', 'ellen', 'degeneres', 'show', '😃']
['i', 'spent', 'twenty', 'minutes', 'herding', 'turkeys', 'by', 'myself', 'so', "that's", 'how', 'my', 'days', 'going', '🦃']
['hello', 'seattle', 'cityscape', '🏗']
['was', 'that', 'a', 'sheep', 'brain', 'in', 'your', 'snap', '👀']
['just', 'waiting', 'for', 'my', 'beard', 'to', 'connect', 'so', 'i', 'can', 'go', 'to', 'my', 'third', 'form', '🙄']
['aye', 'i', 'tried', 'my', 'best', 'to', 'get', 'him', 'out', 'loser', 'dan', '😝']
['day', 'tracking', 'our', 'next', 'ep', 'down', '👏']
['hey', "it's", 'my', 'birthday', "i'm", 'could', 'you', 'say', 'happy', 'birthday', 'flo', '🎉', 'x8']


In [6]:
len(train_data) # число слов

1000000

Эмотиконы представлены в виде символов UNICODE с определенными кодами. Функция ниже позволяет проверить, является ли символ или слово эмотиконом. Будем использовать ее.

In [7]:
# Проверка, является ли слово эмотиконом
import re
emoji_re = re.compile(
    "["
    u"\U00002712\U00002714\U00002716\U0000271d\U00002721\U00002728\U00002733\U00002734\U00002744\U00002747\U0000274c\U0000274e\U00002753-\U00002755\U00002757\U00002763\U00002764\U00002795-\U00002797\U000027a1\U000027b0\U000027bf\U00002934\U00002935\U00002b05-\U00002b07\U00002b1b\U00002b1c\U00002b50\U00002b55\U00003030\U0000303d\U0001f004\U0001f0cf\U0001f170\U0001f171\U0001f17e\U0001f17f\U0001f18e\U0001f191-\U0001f19a\U0001f201\U0001f202\U0001f21a\U0001f22f\U0001f232-\U0001f23a\U0001f250\U0001f251\U0001f300-\U0001f321\U0001f324-\U0001f393\U0001f396\U0001f397\U0001f399-\U0001f39b\U0001f39e-\U0001f3f0\U0001f3f3-\U0001f3f5\U0001f3f7-\U0001f4fd\U0001f4ff-\U0001f53d\U0001f549-\U0001f54e\U0001f550-\U0001f567\U0001f56f\U0001f570\U0001f573-\U0001f579\U0001f587\U0001f58a-\U0001f58d\U0001f590\U0001f595\U0001f596\U0001f5a5\U0001f5a8\U0001f5b1\U0001f5b2\U0001f5bc\U0001f5c2-\U0001f5c4\U0001f5d1-\U0001f5d3\U0001f5dc-\U0001f5de\U0001f5e1\U0001f5e3\U0001f5ef\U0001f5f3\U0001f5fa-\U0001f64f\U0001f680-\U0001f6c5\U0001f6cb-\U0001f6d0\U0001f6e0-\U0001f6e5\U0001f6e9\U0001f6eb\U0001f6ec\U0001f6f0\U0001f6f3\U0001f910-\U0001f918\U0001f980-\U0001f984\U0001f9c0\U00003297\U00003299\U000000a9\U000000ae\U0000203c\U00002049\U00002122\U00002139\U00002194-\U00002199\U000021a9\U000021aa\U0000231a\U0000231b\U00002328\U00002388\U000023cf\U000023e9-\U000023f3\U000023f8-\U000023fa\U000024c2\U000025aa\U000025ab\U000025b6\U000025c0\U000025fb-\U000025fe\U00002600-\U00002604\U0000260e\U00002611\U00002614\U00002615\U00002618\U0000261d\U00002620\U00002622\U00002623\U00002626\U0000262a\U0000262e\U0000262f\U00002638-\U0000263a\U00002648-\U00002653\U00002660\U00002663\U00002665\U00002666\U00002668\U0000267b\U0000267f\U00002692-\U00002694\U00002696\U00002697\U00002699\U0000269b\U0000269c\U000026a0\U000026a1\U000026aa\U000026ab\U000026b0\U000026b1\U000026bd\U000026be\U000026c4\U000026c5\U000026c8\U000026ce\U000026cf\U000026d1\U000026d3\U000026d4\U000026e9\U000026ea\U000026f0-\U000026f5\U000026f7-\U000026fa\U000026fd\U00002702\U00002705\U00002708-\U0000270d\U0000270f]|[#]\U000020e3|[*]\U000020e3|[0]\U000020e3|[1]\U000020e3|[2]\U000020e3|[3]\U000020e3|[4]\U000020e3|[5]\U000020e3|[6]\U000020e3|[7]\U000020e3|[8]\U000020e3|[9]\U000020e3|\U0001f1e6[\U0001f1e8-\U0001f1ec\U0001f1ee\U0001f1f1\U0001f1f2\U0001f1f4\U0001f1f6-\U0001f1fa\U0001f1fc\U0001f1fd\U0001f1ff]|\U0001f1e7[\U0001f1e6\U0001f1e7\U0001f1e9-\U0001f1ef\U0001f1f1-\U0001f1f4\U0001f1f6-\U0001f1f9\U0001f1fb\U0001f1fc\U0001f1fe\U0001f1ff]|\U0001f1e8[\U0001f1e6\U0001f1e8\U0001f1e9\U0001f1eb-\U0001f1ee\U0001f1f0-\U0001f1f5\U0001f1f7\U0001f1fa-\U0001f1ff]|\U0001f1e9[\U0001f1ea\U0001f1ec\U0001f1ef\U0001f1f0\U0001f1f2\U0001f1f4\U0001f1ff]|\U0001f1ea[\U0001f1e6\U0001f1e8\U0001f1ea\U0001f1ec\U0001f1ed\U0001f1f7-\U0001f1fa]|\U0001f1eb[\U0001f1ee-\U0001f1f0\U0001f1f2\U0001f1f4\U0001f1f7]|\U0001f1ec[\U0001f1e6\U0001f1e7\U0001f1e9-\U0001f1ee\U0001f1f1-\U0001f1f3\U0001f1f5-\U0001f1fa\U0001f1fc\U0001f1fe]|\U0001f1ed[\U0001f1f0\U0001f1f2\U0001f1f3\U0001f1f7\U0001f1f9\U0001f1fa]|\U0001f1ee[\U0001f1e8-\U0001f1ea\U0001f1f1-\U0001f1f4\U0001f1f6-\U0001f1f9]|\U0001f1ef[\U0001f1ea\U0001f1f2\U0001f1f4\U0001f1f5]|\U0001f1f0[\U0001f1ea\U0001f1ec-\U0001f1ee\U0001f1f2\U0001f1f3\U0001f1f5\U0001f1f7\U0001f1fc\U0001f1fe\U0001f1ff]|\U0001f1f1[\U0001f1e6-\U0001f1e8\U0001f1ee\U0001f1f0\U0001f1f7-\U0001f1fb\U0001f1fe]|\U0001f1f2[\U0001f1e6\U0001f1e8-\U0001f1ed\U0001f1f0-\U0001f1ff]|\U0001f1f3[\U0001f1e6\U0001f1e8\U0001f1ea-\U0001f1ec\U0001f1ee\U0001f1f1\U0001f1f4\U0001f1f5\U0001f1f7\U0001f1fa\U0001f1ff]|\U0001f1f4\U0001f1f2|\U0001f1f5[\U0001f1e6\U0001f1ea-\U0001f1ed\U0001f1f0-\U0001f1f3\U0001f1f7-\U0001f1f9\U0001f1fc\U0001f1fe]|\U0001f1f6\U0001f1e6|\U0001f1f7[\U0001f1ea\U0001f1f4\U0001f1f8\U0001f1fa\U0001f1fc]|\U0001f1f8[\U0001f1e6-\U0001f1ea\U0001f1ec-\U0001f1f4\U0001f1f7-\U0001f1f9\U0001f1fb\U0001f1fd-\U0001f1ff]|\U0001f1f9[\U0001f1e6\U0001f1e8\U0001f1e9\U0001f1eb-\U0001f1ed\U0001f1ef-\U0001f1f4\U0001f1f7\U0001f1f9\U0001f1fb\U0001f1fc\U0001f1ff]|\U0001f1fa[\U0001f1e6\U0001f1ec\U0001f1f2\U0001f1f8\U0001f1fe\U0001f1ff]|\U0001f1fb[\U0001f1e6\U0001f1e8\U0001f1ea\U0001f1ec\U0001f1ee\U0001f1f3\U0001f1fa]|\U0001f1fc[\U0001f1eb\U0001f1f8]|\U0001f1fd\U0001f1f0|\U0001f1fe[\U0001f1ea\U0001f1f9]|\U0001f1ff[\U0001f1e6\U0001f1f2\U0001f1fc"
    "]+",
    flags=re.UNICODE,
)
def is_emoji(word):
    if emoji_re.match(word[0]):
        return True
    else:
        return False

# Создание и обучение модели векторных представлений

Средствами библиотеки `gensim` создайте и обучите Word2Vec модель на этом наборе слов и словарь всех "слов". Размер векторных представлений - 300, прочие аргументы выбирайте самостоятельно, с учетом последующих требований. Ограничьте время обучения, чтобы оно не превосходило 5 минут. Время обучения выведите на экран.  

In [8]:
!pip install gensim # установка генсима

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 84.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 13.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [10]:
!pip install --upgrade numpy gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.8 MB/s eta 0:00:00


In [8]:
# обучение модели
import gensim.downloader as api
from gensim.models import Word2Vec
from time import time
model = Word2Vec(sentences=None, vector_size=300, workers=4, min_count=10, negative=5, window=5) # создаем модель
t = time()
model.build_vocab(train_data, progress_per=10000) # словарь
print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

Time to build vocab: 0.06 mins


In [9]:
model.corpus_count # число слов в корпусе

1000000

In [10]:
# обучаем модель ~ 10 минут # Заняло 8
t = time()
model.train(train_data, total_examples=model.corpus_count, epochs=20, report_delay=1) # обучаем ~ 10 мин
print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

Time to train the model: 8.07 mins


In [11]:
# сохраним
model.save("word2vec5.bin")
model.wv.save_word2vec_format('word2vec5.txt')

In [12]:
# интерфейс векторных представлений
wv=model.wv

Отберите из словаря только эмотиконы. Они являются символами Unicode. Создайте и выведите на экран таблицу pandas со со столбцами, содержащими эмотиконы и их коды. Код для символа можно получить командой `ord()`. Какой размер таблицы?

In [13]:
# Проходит по всем словам в wv.index_to_key и оставляет только те, которые являются эмодзи
emo_unic_symb = [x for x in wv.index_to_key if is_emoji(x)]

# Преобразование каждого эмодзи в его Unicode-код с помощью функции ord()
emo_unic_code = [ord(x) for x in emo_unic_symb]

# Импорт библиотеки pandas для работы с табличными данными
import pandas as pd

# Создание DataFrame из пар (эмодзи, его Unicode-код)
emo_table = pd.DataFrame(list(zip(emo_unic_symb, emo_unic_code)), columns=['Emoticon', 'Code'])

# Вывод размера таблицы (количество строк и столбцов)
print('Размер таблицы ', emo_table.shape)

# Вывод первых 5 строк таблицы
emo_table.head()

Размер таблицы  (911, 2)


,Emoticon,Code
0,😂,128514
1,😭,128557
2,😍,128525
3,😊,128522
4,🙄,128580


Проверьте, что в словаре помимо эмотиконов есть и обычные слова, выведите на экран 10 первых токенов из словаря.

In [14]:
for index, word in enumerate(wv.index_to_key):
    if index == 10:
        break
    print(f"word #{index}/{len(wv.index_to_key)} is {word}")

word #0/23069 is i
word #1/23069 is the
word #2/23069 is to
word #3/23069 is you
word #4/23069 is a
word #5/23069 is 😂
word #6/23069 is my
word #7/23069 is and
word #8/23069 is me
word #9/23069 is for


# Эксперименты

## 1. Похожесть эмотиконов
Посчитайте и выведите на экран 10 наиболее похожих токенов на эмотикон 😊. Сколько среди них слов, сколько эмотиконов?

*Чек-поинт 1: Модель должна быть обучена так, чтобы среди похожих токенов эмотиконов было не менее 5.*

In [15]:
# Определение функции count_emoji, которая принимает один аргумент sentence
def count_emoji(sentence):
    # Создание списка эмодзи из sentence
    e_count = len([x[0] for x in sentence if is_emoji(x[0])])

    # Возврат кортежа из двух значений
    return e_count, len(sentence) - e_count

In [16]:
# Определяем список эмодзи для поиска похожих слов (в данном случае только смайлик '😊')
query_emo = ['😊']

# Ищем 10 наиболее похожих слов/эмодзи к заданному в query_emo
emo_similar = wv.most_similar(positive=query_emo, topn=10)

# Считаем сколько среди похожих слов является эмодзи, а сколько - обычными словами
emo_count = count_emoji(emo_similar)

# Выводим статистику по количеству эмодзи и слов среди похожих
print('Emoticons \\ words', emo_count)

# Выводим все найденные похожие слова/эмодзи, каждое с новой строки
print(*emo_similar, sep='\n')

Emoticons \ words (10, 0)
('😉', 0.5146317481994629)
('😀', 0.4967454671859741)
('😄', 0.4802841246128082)
('😃', 0.47851893305778503)
('😇', 0.45636817812919617)
('😁', 0.44805020093917847)
('🙂', 0.44519880414009094)
('😌', 0.4389507472515106)
('😆', 0.40508848428726196)
('👍', 0.3790428936481476)


## 2. Похожесть слов на эмотиконы
Посчитайте и выведите на экран 10 наиболее похожих токенов на слово `kitty`. Сколько среди них слов, сколько эмотиконов?

*Чек-поинт 2: Модель должна быть обучена так, чтобы среди похожих токенов был хотя бы 1 эмотикон.*

In [17]:
# Задаем список слов для поиска похожих (в данном случае одно слово 'kitty')
query_word = ['kitty']

# Ищем 10 наиболее семантически близких слов/эмодзи к заданному слову
emo_similar = wv.most_similar(positive=query_word, topn=10)

# Считаем сколько среди похожих слов является эмодзи, а сколько - обычными словами
emo_count = count_emoji(emo_similar)

# Выводим статистику по количеству эмодзи и обычных слов среди похожих
print('Emoticons \\ words', emo_count)

# Выводим все найденные похожие слова/эмодзи, каждое с новой строки
print(*emo_similar, sep='\n')


Emoticons \ words (2, 8)
('kitties', 0.5520135164260864)
('kitten', 0.5004189014434814)
('meow', 0.48224860429763794)
('cat', 0.47981902956962585)
('🐈', 0.4478624761104584)
('caturday', 0.433963418006897)
('pup', 0.4217562973499298)
('🐱', 0.42046046257019043)
('scaredy', 0.4065355658531189)
('pupper', 0.3963871896266937)


Среди похожих токенов из п.2 есть хотя бы 1 эмотикон. Найдите его, для него выведите на экран 5 самых близких токенов. Какой токен оказался самым близким к этому эмотикону, это слово или эмотикон?

In [18]:
# Извлекаем первый эмодзи из списка похожих слов:
emo_first = [x[0] for x in emo_similar if is_emoji(x[0])][0]

# Выводим найденный первый эмодзи
print(emo_first)

# Ищем 5 наиболее похожих слов/эмодзи к найденному эмодзи
emo_similar_first = wv.most_similar(positive=emo_first, topn=5)

# Считаем количество эмодзи и обычных слов среди похожих
emo_count_first = count_emoji(emo_similar_first)

# Выводим статистику по количеству эмодзи и слов
print('Emoticons \\ words', emo_count_first)

# Выводим все найденные похожие слова/эмодзи, каждое с новой строки
print(*emo_similar_first, sep='\n')

# Выводим ближайший элемент (с наибольшим коэффициентом сходства) с указанием его типа:
print('Nearest is a "', emo_similar_first[0][0], '" emoticon' if is_emoji(emo_similar_first[0][0]) else '" word')

🐈
Emoticons \ words (1, 4)
('kitty', 0.44786250591278076)
('cat', 0.43889620900154114)
('🐱', 0.4351957440376282)
('meow', 0.43075278401374817)
('kitties', 0.42537400126457214)
Nearest  is a " kitty " word


## 3. Отношения эмотиконов и слов

### А)
Найдите ближайшее **слово** Х подходящее под уравнение:

🐱 - 🐶 = cat - X

и не совпадающее ни с одним из исходных токенов, входящих в уравнение.

Выведите на экран это слово и уровень похожести.

In [19]:
tokenA='🐱'
tokenB='🐶'
tokenC='cat'

out=wv[tokenC]-wv[tokenA]+wv[tokenB]
candidate=[x for x in wv.similar_by_vector(out,topn=50) if x[0] not in [tokenA,tokenB,tokenC]]
res=[x for x in candidate if not is_emoji(x[0])][0]
print(res)


('dog', 0.5050723552703857)


### Б)
 Найдите ближайший **эмотикон** Х подходящий под уравнение:

kitty - puppy = 🐱 - X  

и не совпадающий ни с одним из исходных токенов, входящих в уравнение.

Выведите на экран этот эмотикон и уровень похожести.

In [20]:
tokenA='kitty'
tokenB='puppy'
tokenC='🐱'

out=wv[tokenC]-wv[tokenA]+wv[tokenB]
candidate=[x for x in wv.similar_by_vector(out,topn=50) if x[0] not in [tokenA,tokenB,tokenC]]
res=[x for x in candidate if is_emoji(x[0])][0]
print(res)



('🐶', 0.30685338377952576)


In [21]:
print(emo_unic_symb)

['😂', '😭', '😍', '😊', '🙄', '😩', '👀', '💀', '😘', '💕', '👍', '🎶', '❤', '👌', '💯', '🙃', '🎉', '🔥', '😒', '👏', '👉', '😉', '😅', '💖', '😎', '😁', '😔', '💋', '😌', '🎃', '💙', '😢', '📷', '😏', '👻', '🎧', '💜', '😴', '😳', '🙏', '🌚', '😑', '😕', '💗', '😜', '👑', '💞', '🙂', '🙌', '😋', '😐', '😈', '😞', '✨', '😪', '🙈', '👇', '💔', '😄', '🌸', '🌹', '😛', '😀', '😬', '💓', '👅', '💛', '🏽', '👊', '😇', '💘', '😫', '🏻', '😱', '😆', '🏼', '🏾', '🌟', '🎈', '💪', '😤', '😡', '🎥', '🎂', '🏆', '😝', '😃', '💚', '🎤', '🍻', '💥', '👈', '🎵', '🌞', '✔', '😷', '😻', '💃', '🍃', '🐶', '🏈', '👋', '🤔', '💰', '🏀', '🔴', '😣', '🐐', '💦', '🎄', '🍾', '😓', '🌎', '😥', '🙊', '📲', '🌊', '😚', '‼', '🌻', '☺', '💫', '🌙', '🍷', '👯', '💎', '🍀', '🔘', '🍕', '😹', '🚨', '🍂', '😶', '➡', '🙁', '🎁', '👄', '😖', '💩', '✌', '🌺', '💍', '🗣', '⚠', '👎', '🐱', '🍁', '👽', '🎯', '💟', '😰', '✅', '🎀', '🎬', '🍒', '🍑', '💤', '📢', '🐾', '💸', '💝', '👂', '🌈', '🎊', '📹', '🍭', '🍸', '🍎', '🐍', '🤘', '▶', '🌴', '📍', '🎼', '🌷', '🎆', '🍹', '👫', '🌠', '😙', '😠', '🏃', '🐻', '❌', '🏡', '🤗', '🍴', '🚮', '❗', '🍦', '🍓', '😟', '🍆', '💅', '🎅', '🍰', '👐', '💨', '😮', '📸',

### В)
 Найдите ближайший **эмотикон** Х подходящий под уравнение:

 man - 😊 = cat - X  

и не совпадающий ни с одним из исходных токенов, входящих в уравнение.

Выведите на экран этот эмотикон и уровень похожести.

In [22]:
tokenA='man'
tokenB='😊' #
#tokenB='😭'
tokenC='cat'
#tokenC='dog'

out=wv[tokenC]-wv[tokenA]+wv[tokenB]
candidate=[x for x in wv.similar_by_vector(out,topn=50) if x[0] not in [tokenA,tokenB,tokenC]]
res=[x for x in candidate if is_emoji(x[0])][0:2]
print(res)


[('🐈', 0.34854239225387573), ('😸', 0.3316681683063507)]


In [24]:
tokenA='heart'
tokenB='💕'
tokenC='fire'


out=wv[tokenC]-wv[tokenA]+wv[tokenB]
candidate=[x for x in wv.similar_by_vector(out,topn=50) if x[0] not in [tokenA,tokenB,tokenC]]
res=[x for x in candidate if is_emoji(x[0])][0:2]
print(res)

[('🔥', 0.29788464307785034), ('👌', 0.2188546061515808)]
